In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
class Scraper:
    def __init__(self, keywords, pages):
        self.keywords = keywords
        self.pages = pages

    def fetch(self, base_url):
        self.base_url = base_url

        self.params = {
            'query': self.keywords,
            'sortby': 'time',
            'page': 2
        }

        self.headers = {
            'sec-ch-ua': '"(Not(A:Brand";v="8", "Chromium";v="99", "Google Chrome";v="99"',
            'sec-ch-ua-platform': "Linux",
            'sec-fetch-site': 'same-origin',
            'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.35 Safari/537.36'
        }

        self.response = requests.get(self.base_url, params=self.params, headers=self.headers)

        return self.response

    def get_articles(self, response):
        article_lists = []

        for page in range(1, int(self.pages)+1):
            url = f"{self.base_url}?query={self.keywords}&sortby=time&page={page}"

            page = requests.get(url)
            soup = BeautifulSoup(page.text, "html.parser")

            articles = soup.find_all("article")

            for article in articles:
                title = article.find("h2", {"class": "title"}).get_text()
                category = article.find("span", {"class": "category"}).get_text()
                published_time = article.find("span", {"class": "date"}).get_text().split(",")[1]
                href = article.find("a")["href"]
                descript = article.find("p").get_text()
                article_lists.append({
                    "title": title, 
                    "category": category, 
                    "published_time": published_time, 
                    "href": href,
                    "descript": descript})

        self.articles = article_lists

        try:
            self.show_results()
        except Exception as e:
            print(e)
        finally:
            print()
            print( "[~] Scraping finished!")
            print(f"[~] Total Articles: {len(self.articles)}")

        return self.articles

    def save_to(self, file_format="csv"):
        '''  '''
        time_scrape = datetime.now().strftime("%m%d%Y_%H%M%S")

        df = pd.DataFrame(self.articles)

        file_name = f"result_{self.keywords}_{time_scrape}"
        if file_format == "csv":
            file_name += ".csv"
            df.to_csv(file_name, index=False)
            print(f"[~] Result saved to '{file_name}'")
        elif file_format == "excel":
            file_name += ".xlsx"
            df.to_excel(file_name, index=False)
            print(f"[~] Result saved to '{file_name}'")

    def show_results(self, row = 5):
        df = pd.DataFrame(self.articles)
        df.index += 1
        if row:
            print(df.head())
        else:
            print(df)
            
if __name__ == '__main__':
    keywords = input("[~] Keywords     : ")
    pages =    input("[~] Total Pages  : ")
    base_url = f"https://www.detik.com/search/searchall"

    scrape = Scraper(keywords, pages)
    response = scrape.fetch(base_url)
    articles = scrape.get_articles(response)

    try:
        ask =             input("[~] Do you want save the results? [y/n]: ").lower()
        if ask == 'y':
            file_format = input("[~] Save to file format? [csv/excel]           : ").lower()
            scrape.save_to(file_format=file_format)
        elif ask == 'n':
            scrape.show_results()
    except Exception as e:
        print(e)
    finally:
        print("[~] Program Finished")

[~] Keywords     : teknologi
[~] Total Pages  : 10
                                               title      category  \
1  TV Analog Mau Dimatikan, Berapa Harga Set Top ...     detikInet   
2  Pengertian Inovasi: Ciri, Manfaat, dan Cara Be...    detikJabar   
3            Urgensi RUU Omnibus Law Sektor Keuangan     detikNews   
4  Rancang Robot Humanoid, Tim ITS Borong Juara I...      detikEdu   
5  Lowongan Kerja Astra Internasional Dibuka, Cek...  detikFinance   

           published_time                                               href  \
1   25 Jul 2022 12:45 WIB  https://inet.detik.com/law-and-policy/d-619730...   
2   25 Jul 2022 12:31 WIB  https://www.detik.com/jabar/bisnis/d-6197336/p...   
3   25 Jul 2022 12:30 WIB  https://news.detik.com/kolom/d-6197216/urgensi...   
4   25 Jul 2022 12:00 WIB  https://www.detik.com/edu/edutainment/d-619710...   
5   25 Jul 2022 11:59 WIB  https://finance.detik.com/lainnya/d-6197261/lo...   

                                            des

In [3]:
data = pd.read_csv('result_teknologi_07252022_130537.csv')
data.head()

,title,category,published_time,href,descript
0,"TV Analog Mau Dimatikan, Berapa Harga Set Top ...",detikInet,25 Jul 2022 12:45 WIB,https://inet.detik.com/law-and-policy/d-619730...,Set Top Box (STB) membuat TV analog dapat mena...
1,"Pengertian Inovasi: Ciri, Manfaat, dan Cara Be...",detikJabar,25 Jul 2022 12:31 WIB,https://www.detik.com/jabar/bisnis/d-6197336/p...,Inovasi diperlukan agar kamu selalu menemukan ...
2,Urgensi RUU Omnibus Law Sektor Keuangan,detikNews,25 Jul 2022 12:30 WIB,https://news.detik.com/kolom/d-6197216/urgensi...,Sektor jasa keuangan saat ini diatur secara se...
3,"Rancang Robot Humanoid, Tim ITS Borong Juara I...",detikEdu,25 Jul 2022 12:00 WIB,https://www.detik.com/edu/edutainment/d-619710...,Tim robotika ITS mengamankan posisi ketiga dal...
4,"Lowongan Kerja Astra Internasional Dibuka, Cek...",detikFinance,25 Jul 2022 11:59 WIB,https://finance.detik.com/lainnya/d-6197261/lo...,Perusahaan menegaskan lowongan kerja Astra ini...


In [4]:
data.shape

(90, 5)

In [6]:
for a in data["href"]:
    print(a)

https://inet.detik.com/law-and-policy/d-6197306/tv-analog-mau-dimatikan-berapa-harga-set-top-box-tv-digital
https://www.detik.com/jabar/bisnis/d-6197336/pengertian-inovasi-ciri-manfaat-dan-cara-berinovasi
https://news.detik.com/kolom/d-6197216/urgensi-ruu-omnibus-law-sektor-keuangan
https://www.detik.com/edu/edutainment/d-6197109/rancang-robot-humanoid-tim-its-borong-juara-internasional-di-thailand
https://finance.detik.com/lainnya/d-6197261/lowongan-kerja-astra-internasional-dibuka-cek-posisi-dan-syaratnya-di-sini
https://news.detik.com/kolom/d-6197163/pemberantasan-korupsi-dan-perampasan-aset
https://inet.detik.com/games-news/d-6197199/beda-dari-minecraft-epic-games-tak-larang-nft-dalam-game
https://wolipop.detik.com/love/d-6197193/trik-tarik-ulur-yang-elegan-saat-pdkt-bikin-gebetan-makin-penasaran
https://news.detik.com/berita/d-6197172/pengertian-haki-ramai-terkait-citayam-fashion-week-jenis-dan-tujuan
https://inet.detik.com/consumer/d-6196982/oppo-reno8-segera-dirilis-di-indonesia

In [7]:
for b in data["descript"]:
    print(b)

Set Top Box (STB) membuat TV analog dapat menangkap siaran TV digital. Berapa harga set top box TV digital dan yang sudah tersertifikasi Kominfo?
Inovasi diperlukan agar kamu selalu menemukan hal baru dan terus berkembang. Pahami juga ciri, manfaat, dan cara berinovasi di sini.
Sektor jasa keuangan saat ini diatur secara sektoral dan tidak menyeluruh, sehingga perlu untuk diatur dalam bentuk perundang-undangan khusus.
Tim robotika ITS mengamankan posisi ketiga dalam kompetisi Robocup 2022 di Thailand Sabtu (16/7). Unik, robot ini bisa bermain sepak bola layaknya atlet.
Perusahaan menegaskan lowongan kerja Astra ini gratis dan tak dipungut biaya apapun.
Melihat perkembangan tindakan terorisme saat ini di Indonesia, semakin penting UU Perampasan Aset Tindak Pidana.
Tim Sweeney, CEO Epic Games, mengungkapkan tidak akan mengikuti jejak Minecraft terkait pelarangan NFT di dalam game. Ini penjelasan yang diberikannya.
Jurus tarik ulur ini tidak sama dengan mempermainkan perasaan orang, ya. J

In [8]:
data["descript"][0]

'Set Top Box (STB) membuat TV analog dapat menangkap siaran TV digital. Berapa harga set top box TV digital dan yang sudah tersertifikasi Kominfo?'